In [1]:
from imutils.video import FPS
import numpy as np
import imutils
import cv2

In [11]:
stream = cv2.VideoCapture("output2.mp4")
fps = FPS().start()

In [21]:
while True:
    (grabbed, frame) = stream.read()

    if not grabbed:
        break
    
    # print frame number
    # crop frame from right 100 px
    print(frame.shape)
    frame = frame[:, :-100, :]
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)
    fps.update()


    

(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 540, 3)
(960, 

KeyboardInterrupt: 

In [17]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

In [18]:
class CentroidTracker():
	
    def __init__(self, maxDisappeared=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            # return early as there are no centroids or tracking info
            # to update
            return self.objects
        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value is at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()
            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]
            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()
            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)
            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        # return the set of trackable objects
        return self.objects

In [19]:
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

In [30]:
ct = CentroidTracker()
(H, W) = (None, None)

net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "model.caffemodel")

In [25]:
import cv2 as cv
import numpy as np

class CustomPythonLayer(cv.dnn.Layer):
    def __init__(self, params, blobs):
        super().__init__(params, blobs)

    def getMemoryShapes(self, inputs):
        # Define the output shape based on the input shapes
        return [inputs[0]]

    def forward(self, inputs, outputs, internals):
        # Perform your custom operation
        outputs[0][:] = np.power(inputs[0], 2)

cv.dnn_registerLayer('CustomLayer', CustomPythonLayer)

In [36]:
vs_bo = cv2.VideoCapture("face_detection_test_video.mp4")
fps_bo = FPS().start()

while True:
    (grabbed, frame) = vs_bo.read()

    if not grabbed:
        break

    frame = frame[:, :-150, :] # retallar per no agafar els aparcats

    frame = imutils.resize(frame, width=400)

    if W is None or H is None:
        (H, W) = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)

    net.setInput(blob)
    detections = net.forward()
    rects = []
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
    # filter out weak detections by ensuring the predicted
    # probability is greater than a minimum threshold
        print(detections[0, 0, i, 2])
        if detections[0, 0, i, 2] > 0.01:
            # compute the (x, y)-coordinates of the bounding box for
            # the object, then update the bounding box rectangles list
            box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
            rects.append(box.astype("int"))
            # draw a bounding box surrounding the object so we can
            # visualize it
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 255, 0), 2)
	# update our centroid tracker using the computed set of bounding
	# box rectangles
    objects = ct.update(rects)
    # loop over the tracked objects
    for (objectID, centroid) in objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# do a bit of cleanup
cv2.destroyAllWindows()
vs_bo.stop()

0.13923182
0.13678814
0.13496202
0.13229418
0.12555395
0.12355563
0.11929814
0.11473168
0.101807475
0.10146136
0.1006369
0.10021753
0.09629441
0.09488378
0.09465143
0.09462568
0.09447925
0.0944454
0.0934978
0.093415834
0.093386725
0.093381554
0.0933355
0.09329515
0.093281336
0.09327374
0.09320637
0.09319371
0.092976406
0.09294657
0.09293361
0.092915036
0.09289032
0.09255648
0.090719126
0.08996413
0.089952685
0.08984294
0.08979616
0.08976433
0.0897274
0.08966911
0.08964985
0.08960199
0.089572065
0.08946909
0.08939179
0.089260496
0.0892331
0.08920906
0.08919691
0.08918128
0.089180276
0.089180194
0.089177944
0.08917787
0.08917736
0.08917635
0.089176014
0.089169346
0.08916667
0.08915785
0.089153975
0.08913225
0.08913086
0.08912797
0.08912659
0.0887281
0.088698305
0.08843406
0.08798814
0.08760495
0.08728609
0.08727666
0.08502683
0.084197946
0.08400791
0.08397512
0.08396719
0.083962746
0.083451055
0.08287491
0.08227537
0.08193364
0.0815968
0.08100957
0.08087731
0.0806171
0.08057034
0.0805111

AttributeError: 'cv2.VideoCapture' object has no attribute 'stop'